data source: Kaggle

### Data Set Explanation:
Welcome! This is a Brazilian ecommerce public dataset of orders made at Olist Store. The dataset has information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil. There are 8 tables in total including: payments, products, orders, items, sellers, customer, geolocation and reviews
Now I would play a role of data analysist of this e-commerce platform company and find some useful insights by exploring this dataset.

### Step 1: Define the questions:
*Sales Performance:
1. What are the total sales revenue and growth rate over a specific period?
2. Which products or categories generate the highest revenue?
3. How do sales vary across different regions or countries?
*Customer Behavior:
1. How frequently do customers make purchases?
2. What is the customer retention rate ?
3. Can we identify any patterns or trends in customer purchase patterns (seasonality, time of day, etc.)?
4. Can we identify any patterns in payment types that they used?
*Product Analysis:
1. Which products have the highest customer ratings and reviews?
*Marketing Effectiveness:
1. Can we measure the customer lifetime value (CLV) and identify high-value customer segments?
*Operational Efficiency:
1. What is the order fulfillment time and how does it impact customer satisfaction?
2. Can we identify any operational bottlenecks or inefficiencies in the supply chain?

### Step 2: Query the neccesary datasets by using SQL
1. What are the total sales revenue and growth rate over a specific period?

SELECT		count(order_id) as total_orders, 
			DATE_PART('month', order_purchase_timestamp) as purchased_month
FROM		ecommerce.orders
GROUP BY	purchased_month
ORDER BY	count(order_id) desc;

*Highest total orders order by month: May, July, March, June, April
*Lowest total orders order by month: September, October, December

SELECT		DATE_PART('year', order_purchase_timestamp) as purchased_year,
			DATE_PART('month', order_purchase_timestamp) as purchased_month,
			sum(price) as total_value, 
			sum(freight_value) as total_freight_value	
FROM		ecommerce.orders
RIGHT JOIN	ecommerce.items ON orders.order_id = items.order_id
GROUP BY	purchased_year,purchased_month
ORDER BY	purchased_year,sum(price) desc;

*May, August and July earn the most, September,October, December earn the least
*Những tháng earn nhiều nhất thường là những tháng 10,11 (vì cái range của dataset có giới hạn), rơi vào khoảng giai đoạn black friday.
*Những tháng earn ít nhất không có trend cụ thể, sẽ phân tích chương trình sale để biết thêm chi tiết

SELECT	
			DATE_PART('year',order_purchase_timestamp) as purchased_year,
			DATE_PART('month', order_purchase_timestamp) as purchased_month,
			count(order_id) as total_orders
FROM		ecommerce.orders
GROUP BY	purchased_year,purchased_month
ORDER BY	purchased_year, purchased_month;

2. Which products or categories generate the highest revenue?

SELECT		product_category_name,COUNT (items.order_id) as total
FROM		ecommerce.orders
RIGHT JOIN	ecommerce.items ON orders.order_id = items.order_id
LEFT JOIN	ecommerce.products ON items.product_id = products.product_id

GROUP BY	product_category_name
ORDER BY	total desc
LIMIT		5;

3. How do sales vary across different regions or countries?
SELECT		count(order_id) as total_orders, geolocation_city, geolocation_lat, geolocation_lng
FROM		ecommerce.orders
LEFT JOIN	ecommerce.customers ON orders.customer_id = customers.customer_id
LEFT JOIN	ecommerce.geolocation ON customers.customer_zip_code_prefix = geolocation.geolocation_zip_code_prefix

GROUP BY	geolocation_city,geolocation_lat, geolocation_lng
ORDER BY	total_orders desc;

**Each city has different lat,lng as well.
Can manage to draw a map distribution

4. Number of transactions based on payment types vs Total value based on payment type

SELECT  count(order_id) as num_of_transactions, payment_type
FROM ecommerce.payments
GROUP BY payment_type;

SELECT  payment_type, sum(payment_value) as total_value 
FROM ecommerce.payments
GROUP BY payment_type
ORDER BY total_value desc;

5. Define good and bad sellers 

SELECT sellers.seller_id, round(avg(review_score),1) as average_score, count(review_score) as total_review
FROM ecommerce.reviews 
LEFT JOIN ecommerce.items
ON reviews."order_id " = items.order_id
LEFT JOIN ecommerce.sellers
ON items.seller_id = sellers.seller_id
GROUP BY sellers.seller_id
HAVING count(review_score) >10
ORDER BY average_score asc;

**can combine with geolocation to mapilize them

6. Delivery operation

SELECT		order_id,
			DATE_PART('day',order_delivered_carrier_date-order_purchase_timestamp) as processing_time,
			DATE_PART('day', order_delivered_customer_date - order_delivered_carrier_date) as delivered_time,
CASE
	WHEN	order_delivered_customer_date > order_estimated_delivery_date THEN 'Late'
	WHEN	order_delivered_customer_date < order_estimated_delivery_date THEN 'Early'
	ELSE	'On time'
END AS	delivery_quality
FROM		ecommerce.orders


7. Define the product category that usually be delivered late

SELECT product_category_name,count(order_id) as total_order
FROM
	(SELECT order_id, items.product_id, product_category_name
	FROM ecommerce.items  
	LEFT JOIN ecommerce.products ON items.product_id = products.product_id
	WHERE order_id in (SELECT order_id from
		(SELECT *
		FROM
			(SELECT		order_id,
						DATE_PART('day',order_delivered_carrier_date-order_purchase_timestamp) as processing_time,
						DATE_PART('day', order_delivered_customer_date - order_delivered_carrier_date) as delivered_time,
			CASE
				WHEN	order_delivered_customer_date > order_estimated_delivery_date THEN 'Late'
				WHEN	order_delivered_customer_date < order_estimated_delivery_date THEN 'Early'
				ELSE	'On time'
			END AS	delivery_quality
			FROM		ecommerce.orders) as innerq1
		WHERE delivery_quality = 'Late') as innerq2)) as inner3
GROUP BY product_category_name
ORDER BY total_order desc
LIMIT 10;

8. Total new customers:  93099. We have TOTAL CUSTOMERS:99441

select customer_unique_id,count(order_id) as num_of_orders
from ecommerce.orders
left join ecommerce.customers on customers.customer_id = orders.customer_id
group by customer_unique_id
having count(order_id) > 1
order by num_of_orders desc

9. New customers every month

SELECT	
		DATE_PART('year',first_time) as first_purchased_year,
		DATE_PART('month', first_time) as first_purchased_month,
		count(customer_unique_id) as new_member
FROM 
    (SELECT distinct customer_unique_id, min(order_purchase_timestamp) as first_time
     FROM ecommerce.orders
     LEFT JOIN ecommerce.customers on customers.customer_id = orders.customer_id
     GROUP BY customer_unique_id) as innerq
GROUP BY first_purchased_year, first_purchased_month
ORDER BY first_purchased_year, first_purchased_month;


### Step 3: Visualize and analyze the insights

Link Tableau: https://public.tableau.com/app/profile/lam.vy.tran/viz/BrazilE-commerce_16851995209700/Dashboard1

Insights:
    - Overall, total orders, and total orders value increase over the time, however, we can notice some peak month such as in May or in November. It can be easily understand that these months fall on the mid-year sale and last-year sale of many brands. However, we can not define that this was the pattern for sale or just a trend in 2017, we should continuosly collect and update data to get a better view.
    - Comparing new customers and retention customers, at this time, the retention rate is very low month on month. On the other hand, the growth rate experienced an increasing trend. 
    - There are some others finding in most popular product 


### Dataset Limitation and Improvement
To provide more accurate and insightful solutions, I think the company should gather more information, particularly as follow:
- Collect customers' backgrounds ( age, gender, occupation, salary) to have a better customers' understandings

Moreover, the datasets should follow some rules to make them easier to analyse.
- The time period we collect data should be a full year, and in a consistent way for easily compare year on year